In [76]:
import regex

In [77]:
special_tokens_list_str = [
    "<|endoftext|>",
    "<|startoftext|>",
    "<|pad|>",
    "<|unk|>",
    "<|mask|>",
]
special_tokens_pattern_str = "|".join(regex.escape(token) for token in special_tokens_list_str)

split_pattern = f'({special_tokens_pattern_str})'
print(special_tokens_pattern_str)
print(split_pattern)

<\|endoftext\|>|<\|startoftext\|>|<\|pad\|>|<\|unk\|>|<\|mask\|>
(<\|endoftext\|>|<\|startoftext\|>|<\|pad\|>|<\|unk\|>|<\|mask\|>)


In [78]:
special_token = ["<|endoftext|>"]
chunk_bytes = special_token[0].encode("utf-8").join([
    b"Hello world!",
    b"Python is great.",
    b"Test chunking.",
    b"End of file."
])
chunk_bytes

b'Hello world!<|endoftext|>Python is great.<|endoftext|>Test chunking.<|endoftext|>End of file.'

In [84]:
chunk_text = chunk_bytes.decode("utf-8", errors="ignore")

# Regex just support str
special_token_str = special_token[0]
special_escape_token = f'({regex.escape(special_token_str)})'

segments = regex.split(special_escape_token, chunk_text) # regex.split(str, str)

byte_lst: list[list[bytes]] = []


for i in range(len(segments)):
    if segments[i] == special_token_str:
        byte_lst.append([special_token_str.encode("utf-8")])
    else:
        PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
        for match in regex.finditer(PAT, segments[i]):
            match_str = match.group(0)
            byte_lst.append([bytes([c]) for c in match_str.encode("utf-8")])

byte_lst

[[b'H', b'e', b'l', b'l', b'o'],
 [b' ', b'w', b'o', b'r', b'l', b'd'],
 [b'!'],
 [b'<|endoftext|>'],
 [b'P', b'y', b't', b'h', b'o', b'n'],
 [b' ', b'i', b's'],
 [b' ', b'g', b'r', b'e', b'a', b't'],
 [b'.'],
 [b'<|endoftext|>'],
 [b'T', b'e', b's', b't'],
 [b' ', b'c', b'h', b'u', b'n', b'k', b'i', b'n', b'g'],
 [b'.'],
 [b'<|endoftext|>'],
 [b'E', b'n', b'd'],
 [b' ', b'o', b'f'],
 [b' ', b'f', b'i', b'l', b'e'],
 [b'.']]

In [80]:
bytes([97]) + bytes([98])

b'ab'

In [81]:
from collections import defaultdict

vocab_size = 50
vocab_lst = []
merge_lst = []

for _ in range(len(vocab_lst), vocab_size):
    count = defaultdict(int)
    # Find the most frequent byte pair in the tokens_lst
    for it in tokens_lst:
        for i in range(len(it) - 1):
            pair = (it[i], it[i + 1])
            count[pair] += 1
    if not count:
        break
    max_pair = max(count, key=count.get)

    new_vocab = max_pair[0] + max_pair[1]
    vocab_lst.append(new_vocab)
    merge_lst.append(max_pair)

    new_tokens_lst = []
    for it in tokens_lst:
        i = 0
        new_token = []
        while i < len(it):
            if i < len(it) - 1 and (it[i], it[i + 1]) == max_pair:
                new_token.append(new_vocab)
                i += 2
            else:
                new_token.append(it[i])
                i += 1
        new_tokens_lst.append(new_token)

    tokens_lst = new_tokens_lst

vocab_lst

[]